In [1]:
from datasets import load_dataset

ds = load_dataset("DREAM_training_dataset")

In [2]:
ds = ds.shuffle(seed=42)

In [3]:
ds['train'][0]

{'dataset': 'moral_stories',
 'id': 'moral_stories_training_1783',
 'question': "[SITUATION] Alphonse got assigned to a group of people he didn't like for a school project. Alphonse tries his best to work with his group and helps to submit the project. [QUERY] consequence",
 'answer': "[moral_consequence] Everyone in his group get's an A, and they actually end up finding stuff in common and becoming friends."}

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import os 
os.environ['TOKENIZERS_PARALLELISM'] = "false"
os.environ["WANDB_DISABLED"] = "true"

checkpoint = "t5-small"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [5]:
def preprocess_dataset(examples):
    model_inputs = tokenizer(examples['question'])
    labels = tokenizer(examples['answer'])
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [6]:
tokenized_ds = ds.map(preprocess_dataset, batched=True)
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['dataset', 'id', 'question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 78286
    })
    validation: Dataset({
        features: ['dataset', 'id', 'question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 15000
    })
    test: Dataset({
        features: ['dataset', 'id', 'question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 20441
    })
})

In [7]:
import evaluate
rouge = evaluate.load("rouge")

import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [8]:
from huggingface_hub import notebook_login

notebook_login()

In [9]:
training_args = Seq2SeqTrainingArguments(
    output_dir="DREAM-t5-small",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=8,
    load_best_model_at_end=True,
    eval_accumulation_steps=8,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["test"],
    eval_dataset=tokenized_ds["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


: 

In [ ]:
trainer.push_to_hub()